In [19]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "PLrgwD9TleU"

ytt_api = YouTubeTranscriptApi()
fetched_transcript = ytt_api.fetch(video_id)

In [20]:
full_text = ""
for snippet in fetched_transcript:
    full_text += snippet.text + " "
print(full_text)

in this video we're going to focus on how to graph conic sections like ellipses parabas hyperbolas and also circles as well and in addition to that we're going to talk about how to look at equation um and to tell if it's going to be a circle ellipse Parabola and hyperbola and how to put it in standard form so let's focus on the uh graphing part let's say if we have this equation x^2 + y^2 = 9 and this is a circle but how can we graph this circle so the standard equation for a circle it's it's x - h^ 2 + y - k^ 2 is equal to R 2 the center of the circle is H comma K but there's no H and K in this uh problem so the center is based on the origin it's 0 0 now R 2 is basically 9 so if r s is equal to 9 if you take the square root of both sides um you'll see that the radius is three so starting from the origin which is 0 0 you need to go three to the right three to the left up three and then down three and then draw a circle connecting those points so that's how you can graph uh that particu

In [21]:
transcript_prompt = f"""
You are a teacher creating quizzes from a lecture transcript. Using the text below, generate a quiz with a question for each important topic. The quiz should include **multiple-choice questions (MCQs)** only. Each question should have:

1. A "question" string.
2. An "options" list with exactly 4 options.
3. An "answer" string indicating the correct option.
4. A "difficulty" string which can be "easy", "medium", or "hard".

Output the quiz strictly in JSON format like this:

[
  {{
    "question": "Example question?",
    "options": ["Option A", "Option B", "Option C", "Option D"],
    "answer": "Option B",
    "difficulty": "medium"
  }},
  ...
]

**Transcript:**
"{full_text}"
"""

In [9]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-4B-Instruct-2507"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # for better accuracy
    bnb_4bit_use_double_quant=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [22]:
prompt = transcript_prompt
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)

thinking content: 


In [23]:
import re, json

content = re.sub(r'[^\x00-\x7F]+', '', content)  # removes non-ASCII chars

print("content:", content)

content: [
  {
    "question": "What is the standard form of a circle's equation and what does the center (h, k) represent?",
    "options": [
      "x - h^2 + y - k^2 = r^2; the center (h, k) represents the origin",
      "x^2 + y^2 = r^2; the center (h, k) represents the vertex",
      "x - h^2 + y - k^2 = r^2; the center (h, k) represents the center of the circle",
      "x^2 + y^2 = 1; the center (h, k) represents the focus"
    ],
    "answer": "x - h^2 + y - k^2 = r^2; the center (h, k) represents the center of the circle",
    "difficulty": "easy"
  },
  {
    "question": "For the equation x^2/25 + y^2/49 = 1, what is the major axis and why?",
    "options": [
      "The major axis is horizontal because 25 > 49",
      "The major axis is vertical because 49 > 25",
      "The major axis is horizontal because a^2 is under y^2",
      "The major axis is horizontal because a^2 is under x^2"
    ],
    "answer": "The major axis is vertical because 49 > 25",
    "difficulty": "medium"

In [24]:
import json

# 'content' is the string returned by the LLM
try:
    quiz = json.loads(content)  # parse the JSON string
except json.JSONDecodeError:
    print("Failed to parse JSON. Here's the raw content:")
    print(content)
else:
    # nicely print each question
    for i, q in enumerate(quiz, start=1):
        print(f"Question {i}: {q['question']}")
        for j, option in enumerate(q['options'], start=1):
            print(f"  {j}. {option}")
        print(f"Answer: {q['answer']}")
        print(f"Difficulty: {q['difficulty']}")
        print("-" * 50)

Question 1: What is the standard form of a circle's equation and what does the center (h, k) represent?
  1. x - h^2 + y - k^2 = r^2; the center (h, k) represents the origin
  2. x^2 + y^2 = r^2; the center (h, k) represents the vertex
  3. x - h^2 + y - k^2 = r^2; the center (h, k) represents the center of the circle
  4. x^2 + y^2 = 1; the center (h, k) represents the focus
Answer: x - h^2 + y - k^2 = r^2; the center (h, k) represents the center of the circle
Difficulty: easy
--------------------------------------------------
Question 2: For the equation x^2/25 + y^2/49 = 1, what is the major axis and why?
  1. The major axis is horizontal because 25 > 49
  2. The major axis is vertical because 49 > 25
  3. The major axis is horizontal because a^2 is under y^2
  4. The major axis is horizontal because a^2 is under x^2
Answer: The major axis is vertical because 49 > 25
Difficulty: medium
--------------------------------------------------
Question 3: For the hyperbola x^2/9 - y^2/4 = 1